In [1]:
import copy
import itertools
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

project_root = Path.cwd().parent
sys.path.append(str(project_root))

from src.utils.utils_data import create_train_test_data
from src.utils.utils_models import build_model
from src.utils.utils_pipeline import set_global_seed

In [2]:
BASE_JOB = {
    "experiment_name": "hyperparameter_tuning",
    "model_runs": 1,
    "seed": 42,
    "function": ["f_sinusoidal_hc2_1_feat", "sigma_cubic_hc2_1_feat"],
    "train_interval": [-4.0, 4.0],
    "train_instances": 1100, # 100 for hyperparameter tuning
    "train_repeats": 0,
    "test_interval": [-6.0, 6.0],
    "test_grid_length": 1000,
    "test_add_points": None, # not needed for hyperparameter tuning
    "model_name": None, # filled by specific model
    "model_params": {
        "hidden_features": [64, 64],
        "activations": ["relu", "relu"],
        "weight_decay": 0.0,
        "p_drop": 0.0,
        "shuffle": True,
        "device": "cpu",
        "target_dim": 1,
        }
}

def make_job(base_job: dict, model_name: str, model_params: dict) -> dict:
    """
    Creates a new job dictionary based on a shared base job configuration.
    Merges BASE_JOB['model_params'] with model-specific params (model_params overrides).
    """
    job = copy.deepcopy(base_job)
    job["model_name"] = model_name

    # Merge: base defaults first, then model-specific overrides
    merged_params = copy.deepcopy(base_job["model_params"])
    merged_params.update(copy.deepcopy(model_params))
    job["model_params"] = merged_params
    return job



def grid_search(job: dict, param_grid: dict, train_size: int, val_size: int):
    """Runs grid search over specified hyperparameters.

    Args:
      job: config dict.
      param_grid: parameter grid to test.

    Returns:
      results_df: pd.DataFrame with one row per hyperparameter combo and its val_nll. Sorted by val_nll.
    """
    grid_job = copy.deepcopy(job)
    df_train, _, n_features = create_train_test_data(grid_job)

    # Split into train/val
    df_train, df_val = train_test_split(df_train, train_size=train_size, test_size=val_size,
                                        random_state=grid_job["seed"])

    x_cols = df_train.columns[df_train.columns.str.startswith("x")]
    X_train = df_train.loc[:, x_cols].to_numpy(dtype=np.float32)
    y_train = df_train["y"].to_numpy(dtype=np.float32)

    X_val = df_val.loc[:, x_cols].to_numpy(dtype=np.float32)
    y_val = df_val["y"].to_numpy(dtype=np.float32)

    # Normalize y_train (no need to normalize y_val, because predictions are denormalized 
    # and val_nll is based on denormalized predictions)
    y_mean = y_train.mean(axis=0)
    y_std = y_train.std(axis=0)
    y_train_norm = (y_train - y_mean) / y_std

    # Create parameter grid
    keys = list(param_grid.keys())
    values_product = list(itertools.product(*(param_grid[k] for k in keys)))
    n_trials = len(values_product)

    results = []
    best_val_nll = float("inf")
    best_combo = None

    print(f"[GridSearch] Trials: {n_trials} over keys {keys}")

    for i, vals in enumerate(values_product, start=1):
        # Seeding per Trial
        trial_job = copy.deepcopy(grid_job)
        set_global_seed(trial_job["seed"])

        # Update model_params with current grid parameters
        combo = dict(zip(keys, vals))
        trial_params = copy.deepcopy(trial_job["model_params"])
        trial_params.update(combo)
        trial_job["model_params"] = trial_params

        print(f"[{i:>3}/{n_trials}] model_params -> {trial_job['model_params']}")

        model = build_model(trial_job["model_name"], trial_job["model_params"], trial_job["seed"], n_features)

        model.fit(X_train, y_train_norm)

        # Validate
        _, _, _, val_nll, _ = model.predict_with_uncertainties(
            X_test=X_val, y_test=y_val, y_mean=y_mean, y_std=y_std, test_interval=trial_job["test_interval"]
        )
        val_nll = float(val_nll.item())
        row = {**combo,
               "val_nll": val_nll}
        results.append(row)
        print(f"    -> val_nll={val_nll:.6f}")

        if val_nll < best_val_nll:
            best_val_nll = val_nll
            best_combo = combo

    results_df = pd.DataFrame(results).sort_values("val_nll", ascending=True).reset_index(drop=True)

    print("\n[Best]")
    print(best_combo, "-> val_nll=", best_val_nll)

    return results_df

## Bayes by Backprop

In [3]:
bbb_params = {
    "prior_mean": 0.0,
    "n_mc_samples": 300,
    "mu_init_mean": 0.0,
    # grid search
    "prior_sigma": None,
    "kl_beta": None,
    "mu_init_std": None,
    "rho_init": None,
    "lr": None,
    "epochs": None,
    "batch_size": None
}

bbb_job = make_job(BASE_JOB, model_name="bbb", model_params=bbb_params)

results_df= grid_search(bbb_job, 
                        param_grid = {
                        "rho_init":   [-3.0, -4.0, -5.0],
                        "kl_beta":    [0.3, 0.6, 1.0],
                        "prior_sigma":[0.5, 1.0, 2.0],
                        "mu_init_std":[0.1, 0.2],
                        "lr": [0.001, 0.01],
                        "epochs": [100, 200],
                        "batch_size": [32, 64],
                        },
                        train_size=1000,
                        val_size=100
                        )
results_df

[GridSearch] Trials: 432 over keys ['rho_init', 'kl_beta', 'prior_sigma', 'mu_init_std', 'lr', 'epochs', 'batch_size']
[  1/432] model_params -> {'hidden_features': [64, 64], 'activations': ['relu', 'relu'], 'weight_decay': 0.0, 'p_drop': 0.0, 'shuffle': True, 'device': 'cpu', 'target_dim': 1, 'prior_mean': 0.0, 'n_mc_samples': 300, 'mu_init_mean': 0.0, 'prior_sigma': 0.5, 'kl_beta': 0.3, 'mu_init_std': 0.1, 'rho_init': -3.0, 'lr': 0.001, 'epochs': 100, 'batch_size': 32}
    -> val_nll=-2.642717
[  2/432] model_params -> {'hidden_features': [64, 64], 'activations': ['relu', 'relu'], 'weight_decay': 0.0, 'p_drop': 0.0, 'shuffle': True, 'device': 'cpu', 'target_dim': 1, 'prior_mean': 0.0, 'n_mc_samples': 300, 'mu_init_mean': 0.0, 'prior_sigma': 0.5, 'kl_beta': 0.3, 'mu_init_std': 0.1, 'rho_init': -3.0, 'lr': 0.001, 'epochs': 100, 'batch_size': 64}
    -> val_nll=-2.464428
[  3/432] model_params -> {'hidden_features': [64, 64], 'activations': ['relu', 'relu'], 'weight_decay': 0.0, 'p_drop

,rho_init,kl_beta,prior_sigma,mu_init_std,lr,epochs,batch_size,val_nll
0,-5.0,0.3,0.5,0.1,0.001,100,32,-2.772500
1,-5.0,0.6,1.0,0.1,0.001,100,32,-2.772050
2,-5.0,0.6,2.0,0.1,0.001,100,32,-2.772004
3,-5.0,0.3,2.0,0.1,0.001,100,32,-2.771509
4,-5.0,0.3,1.0,0.2,0.001,100,32,-2.771157
...,...,...,...,...,...,...,...,...
427,-5.0,1.0,2.0,0.1,0.010,100,32,NaN
428,-5.0,1.0,2.0,0.1,0.010,200,32,NaN
429,-5.0,1.0,2.0,0.2,0.010,100,32,NaN
430,-5.0,1.0,2.0,0.2,0.010,200,32,NaN


# Deep Ensemble

In [4]:
ensemble_params = {
    "n_members": 5,
    # grid search
    "weight_decay": None,
    "lr": None,
    "epochs": None,
    "batch_size": None
}

ensemble_job = make_job(BASE_JOB, model_name="ensemble", model_params=ensemble_params)

results_df= grid_search(ensemble_job, 
                        param_grid = {
                        "weight_decay": [1e-3, 5e-4, 1e-4],
                        "lr": [0.001, 0.01],
                        "epochs": [100, 200],
                        "batch_size": [32, 64],
                        },
                        train_size=1000,
                        val_size=100
                        )
results_df

[GridSearch] Trials: 24 over keys ['weight_decay', 'lr', 'epochs', 'batch_size']
[  1/24] model_params -> {'hidden_features': [64, 64], 'activations': ['relu', 'relu'], 'weight_decay': 0.001, 'p_drop': 0.0, 'shuffle': True, 'device': 'cpu', 'target_dim': 1, 'n_members': 5, 'lr': 0.001, 'epochs': 100, 'batch_size': 32}
    -> val_nll=-2.766841
[  2/24] model_params -> {'hidden_features': [64, 64], 'activations': ['relu', 'relu'], 'weight_decay': 0.001, 'p_drop': 0.0, 'shuffle': True, 'device': 'cpu', 'target_dim': 1, 'n_members': 5, 'lr': 0.001, 'epochs': 100, 'batch_size': 64}
    -> val_nll=-2.769816
[  3/24] model_params -> {'hidden_features': [64, 64], 'activations': ['relu', 'relu'], 'weight_decay': 0.001, 'p_drop': 0.0, 'shuffle': True, 'device': 'cpu', 'target_dim': 1, 'n_members': 5, 'lr': 0.001, 'epochs': 200, 'batch_size': 32}
    -> val_nll=-2.777061
[  4/24] model_params -> {'hidden_features': [64, 64], 'activations': ['relu', 'relu'], 'weight_decay': 0.001, 'p_drop': 0.0, '

,weight_decay,lr,epochs,batch_size,val_nll
0,0.0001,0.001,200,32,-2.779435
1,0.0005,0.001,200,32,-2.778190
2,0.0001,0.001,200,64,-2.777538
3,0.0010,0.010,200,32,-2.777305
4,0.0001,0.010,200,32,-2.777282
5,0.0010,0.001,200,32,-2.777061
6,0.0005,0.001,200,64,-2.776570
7,0.0010,0.001,200,64,-2.776086
8,0.0005,0.010,200,64,-2.770862
9,0.0005,0.001,100,64,-2.770028


# MC Dropout

In [5]:
mcdropout_params = {
        'n_mc_samples': 300,
        # grid search
        'p_drop': None,
        'weight_decay': None,
        }

mcdropout_job = make_job(BASE_JOB, model_name="mcdropout", model_params=mcdropout_params)
print(mcdropout_job)

results_df= grid_search(mcdropout_job, 
                        param_grid = {
                        "weight_decay": [1e-3, 5e-4, 1e-4],
                        "p_drop": [0.05, 0.1, 0.2],
                        "lr": [0.001, 0.01],
                        "epochs": [100, 200],
                        "batch_size": [32, 64],
                        },
                        train_size=1000,
                        val_size=100
                        )
results_df

{'experiment_name': 'hyperparameter_tuning', 'model_runs': 1, 'seed': 42, 'function': ['f_sinusoidal_hc2_1_feat', 'sigma_cubic_hc2_1_feat'], 'train_interval': [-4.0, 4.0], 'train_instances': 1100, 'train_repeats': 0, 'test_interval': [-6.0, 6.0], 'test_grid_length': 1000, 'test_add_points': None, 'model_name': 'mcdropout', 'model_params': {'hidden_features': [64, 64], 'activations': ['relu', 'relu'], 'weight_decay': None, 'p_drop': None, 'shuffle': True, 'device': 'cpu', 'target_dim': 1, 'n_mc_samples': 300}}
[GridSearch] Trials: 72 over keys ['weight_decay', 'p_drop', 'lr', 'epochs', 'batch_size']
[  1/72] model_params -> {'hidden_features': [64, 64], 'activations': ['relu', 'relu'], 'weight_decay': 0.001, 'p_drop': 0.05, 'shuffle': True, 'device': 'cpu', 'target_dim': 1, 'n_mc_samples': 300, 'lr': 0.001, 'epochs': 100, 'batch_size': 32}
    -> val_nll=-2.765452
[  2/72] model_params -> {'hidden_features': [64, 64], 'activations': ['relu', 'relu'], 'weight_decay': 0.001, 'p_drop': 0.0

,weight_decay,p_drop,lr,epochs,batch_size,val_nll
0,0.0001,0.05,0.010,100,32,-2.775205
1,0.0010,0.05,0.001,200,64,-2.772406
2,0.0005,0.05,0.001,200,64,-2.771563
3,0.0001,0.10,0.001,200,32,-2.771299
4,0.0001,0.05,0.001,200,64,-2.767868
...,...,...,...,...,...,...
67,0.0001,0.20,0.010,200,32,-2.667511
68,0.0010,0.20,0.010,200,32,-2.667184
69,0.0005,0.10,0.010,100,64,-2.659669
70,0.0001,0.20,0.010,100,32,-2.653566


# Deep Evidential Regression

In [6]:

evidential_params = {
        # grid search
        'coeff': None,
        'lr': None,
        'epochs': None,
        'batch_size': None
        }

evidential_job = make_job(BASE_JOB, model_name="evidential", model_params=evidential_params)

results_df= grid_search(evidential_job, 
                        param_grid = {
                        "coeff": [0.001, 0.005, 0.01, 0.05, 0.1],
                        "lr": [0.001, 0.01],
                        "epochs": [100, 200],
                        "batch_size": [32, 64],
                        },
                        train_size=1000,
                        val_size=100
                        )
results_df

[GridSearch] Trials: 40 over keys ['coeff', 'lr', 'epochs', 'batch_size']
[  1/40] model_params -> {'hidden_features': [64, 64], 'activations': ['relu', 'relu'], 'weight_decay': 0.0, 'p_drop': 0.0, 'shuffle': True, 'device': 'cpu', 'target_dim': 1, 'coeff': 0.001, 'lr': 0.001, 'epochs': 100, 'batch_size': 32}
    -> val_nll=-2.779688
[  2/40] model_params -> {'hidden_features': [64, 64], 'activations': ['relu', 'relu'], 'weight_decay': 0.0, 'p_drop': 0.0, 'shuffle': True, 'device': 'cpu', 'target_dim': 1, 'coeff': 0.001, 'lr': 0.001, 'epochs': 100, 'batch_size': 64}
    -> val_nll=-2.703464
[  3/40] model_params -> {'hidden_features': [64, 64], 'activations': ['relu', 'relu'], 'weight_decay': 0.0, 'p_drop': 0.0, 'shuffle': True, 'device': 'cpu', 'target_dim': 1, 'coeff': 0.001, 'lr': 0.001, 'epochs': 200, 'batch_size': 32}
    -> val_nll=-2.775054
[  4/40] model_params -> {'hidden_features': [64, 64], 'activations': ['relu', 'relu'], 'weight_decay': 0.0, 'p_drop': 0.0, 'shuffle': True,

,coeff,lr,epochs,batch_size,val_nll
0,0.005,0.010,200,32,-2.783984
1,0.001,0.010,200,32,-2.782626
2,0.010,0.001,200,64,-2.781754
3,0.001,0.010,100,64,-2.780627
4,0.010,0.001,100,32,-2.779998
5,0.001,0.001,200,64,-2.779904
6,0.005,0.010,100,64,-2.779893
7,0.001,0.001,100,32,-2.779688
8,0.005,0.001,200,64,-2.779436
9,0.010,0.010,200,32,-2.778340
